In [ ]:
import os
from RZutilpy.system import Path, unix_wrapper, gettimestr, makedirs
from RZutilpy.rzio import matchfiles
import scipy.io as scio
import re
import pandas as pd
from pathlib import Path

In [ ]:
#加载mask
%cd /home/data/yufengspace/23BPhappy/results/



In [ ]:
folders = matchfiles(f'/home/data/yufengspace/23BPhappy/*_BPhappy_*')
Param1 = 'option_CRandEV'
Param2 = 'gamble_RPE'
seed = 'lvs' #set seedlabel
AFNI_HOME = os.getenv('AFNIDIR')
MNIT1 = AFNI_HOME + '/MNI152_2009_template_SSW.nii.gz'

for folderofsubj in folders:
    subj = folderofsubj[-4:]

    ROI_dir = Path(f'{folderofsubj}/funcROI/')
    ROI_dir = Path(ROI_dir)
    #assert not GLM_dir.exists(), f'GLM dir {GLM_dir} already exists'
    #verify that the results directory does not yet exist
    if not os.path.exists(ROI_dir):
        os.makedirs(ROI_dir)

    os.chdir(ROI_dir)   

    %cd $ROI_dir
    ! 3dcalc -a /home/data/yufengspace/23BPhappy/results/clust2_lvs_mask_0002+tlrc -b ../funcppi/mask.{subj}.{seed}.nii.gz -expr 'ispositive(a) * ispositive(b)' -prefix cluster2in_{seed}_mask
    ! 3dmaskave  -mask cluster2in_{seed}_mask+tlrc ../funcGLM1/{subj}_{Param1}_{Param2}_stats+tlrc'[12]' > GLM2_{subj}_cluster2in_{seed}_{Param2}_avgbeta.txt


In [ ]:
Group = scio.loadmat('/home/data/yufengspace/23BPhappy/temp/clidata.mat')
Group = Group['clidata']



GroupMDD = []
GroupBD = []

for i in range(0,len(Group)):
    subj = Group[i][0][0]

    if Group[i][10][0][0] == 0:
        GroupMDD.append(subj)
    elif Group[i][10][0][0] == 1:
        GroupBD.append(subj)

    #statsfiles = matchfiles(f'{GLM_dir}/*{Param1}_{Param2}_stats+tlrc.BRIK')
    #!cp {statsfiles}  /home/yufengxia/Data/23BPhappy/temp
    #print(statsfiles)

In [ ]:
output_file = f'GLM2_cluster2in_{seed}_{Param2}_avgbeta.xlsx'
# 用于存储所有数字的列表
numbers_list = []

folders = matchfiles(f'/home/data/yufengspace/23BPhappy/*_BPhappy_*')
for folderofsubj in folders:
    subj = folderofsubj[-4:]
    if subj in GroupMDD or subj in GroupBD:
        ROI_dir = Path(f'{folderofsubj}/funcROI/')
        ROI_dir = Path(ROI_dir)
        
        
        %cd $ROI_dir
        txt_file = f'GLM2_{subj}_cluster2in_{seed}_{Param2}_avgbeta.txt'
        with open(txt_file, 'r') as file:
            # 读取文件内容
            content = file.read()
            # 使用正则表达式匹配数字
            match = re.search(r'-?\d+(\.\d+)?(e[-+]?\d+)?', content)
            if match:
                # 将匹配到的数字添加到列表中
                numbers_list.append(match.group(0))

%cd /home/data/yufengspace/23BPhappy/results/
df = pd.DataFrame(numbers_list, columns=['Beta_Weights'])
# 将DataFrame写入Excel文件
df.to_excel(output_file, index=False)



In [ ]:

folders = matchfiles(f'/home/data/yufengspace/23BPhappy/*_BPhappy_*')
for folderofsubj in folders:
    subj = folderofsubj[-4:]
    ROI_dir = Path(f'{folderofsubj}/funcROI/')
    ROI_dir = Path(ROI_dir)
    # verify that the results directory does not yet exist
    ! rm -rf {ROI_dir}
